# Capstone Project OECD Cancer Data and Machine Learning

### Import Necessary Libraries

In [63]:
import pandas as pd
import numpy as np

### Read CSV and Determine Column Names for Cleaning

In [64]:
OECD = pd.read_csv('OECDHEALTHDATA.csv')

print(OECD.columns)

Index(['STRUCTURE', 'STRUCTURE_ID', 'STRUCTURE_NAME', 'ACTION', 'REF_AREA',
       'Reference area', 'FREQ', 'Frequency of observation', 'MEASURE',
       'Measure', 'UNIT_MEASURE', 'Unit of measure', 'AGE', 'Age', 'SEX',
       'Sex', 'SOCIO_ECON_STATUS', 'Socio-economic status', 'DEATH_CAUSE',
       'Cause of death', 'CALC_METHODOLOGY', 'Calculation methodology',
       'GESTATION_THRESHOLD', 'Gestation period threshold', 'HEALTH_STATUS',
       'Health status', 'DISEASE', 'Disease', 'CANCER_SITE', 'Cancer site',
       'TIME_PERIOD', 'Time period', 'OBS_VALUE', 'Observation value',
       'DECIMALS', 'Decimals', 'OBS_STATUS', 'Observation status',
       'OBS_STATUS2', 'Observation status 2', 'OBS_STATUS3',
       'Observation status 3', 'UNIT_MULT', 'Unit multiplier'],
      dtype='object')


### Drop unecessary columns

In [65]:
# drop features
OECD_cleaned = OECD.drop(['STRUCTURE', 'STRUCTURE_ID', 'STRUCTURE_NAME', 'ACTION', 'REF_AREA', 'FREQ', 'Frequency of observation', 'MEASURE',
       'Measure', 'UNIT_MEASURE', 'Unit of measure', 'AGE', 'Age', 'SEX', 'SOCIO_ECON_STATUS', 'Socio-economic status', 'DEATH_CAUSE',
       'Cause of death', 'CALC_METHODOLOGY', 'Calculation methodology',
       'GESTATION_THRESHOLD', 'Gestation period threshold', 'HEALTH_STATUS',
       'Health status', 'DISEASE', 'Disease', 'CANCER_SITE','Time period', 'Observation value',
       'DECIMALS', 'Decimals', 'OBS_STATUS', 'Observation status',
       'OBS_STATUS2', 'Observation status 2', 'OBS_STATUS3',
       'Observation status 3', 'UNIT_MULT', 'Unit multiplier'], axis=1)

# rename features
OECD_cleaned = OECD_cleaned.rename(columns={'OBS_VALUE' : 'Cancer Rate', 'TIME_PERIOD' : 'Year', 'Reference area' : 'Country'})


# Print the result

#check that correct columns were dropped
OECD_cleaned.head(10)

,Country,Sex,Cancer site,Year,Cancer Rate
0,Slovak Republic,Female,Malignant neoplasms of female breast,2002,48.0
1,Slovak Republic,Female,Malignant neoplasms of female breast,2008,53.4
2,Slovak Republic,Female,Malignant neoplasms of female breast,2012,57.5
3,Iceland,Male,"Malignant neoplasms of trachea, bronchus, lung",2002,30.0
4,Iceland,Male,"Malignant neoplasms of trachea, bronchus, lung",2008,29.3
5,Iceland,Male,"Malignant neoplasms of trachea, bronchus, lung",2012,31.0
6,Sweden,Female,Malignant neoplasms of female breast,2000,81.0
7,Sweden,Female,Malignant neoplasms of female breast,2002,87.8
8,Sweden,Female,Malignant neoplasms of female breast,2008,82.7
9,Sweden,Female,Malignant neoplasms of female breast,2012,80.4


In [66]:
#determine the range of time covered

year_values = OECD_cleaned['Year'].unique()
min_year = min(year_values)
max_year = max(year_values)

#The countires included

countries = OECD_cleaned['Country'].unique()

#Types of cancer

cancers = OECD_cleaned['Cancer site'].unique()

#print resluts

print(f"Range of Years: {min_year} to {max_year}")
print(countries)
print(cancers)

Range of Years: 2000 to 2012
['Slovak Republic' 'Iceland' 'Sweden' 'Austria' 'France' 'Colombia'
 'Estonia' 'United Kingdom' 'Korea' 'Italy' 'Netherlands' 'Brazil'
 'Indonesia' 'United States' 'Czechia' 'Australia' 'Luxembourg' 'Canada'
 'Denmark' 'South Africa' 'Ireland' 'Costa Rica' 'India' 'Lithuania'
 'Hungary' 'Türkiye' 'Belgium' 'Spain' 'Russia' 'Latvia' 'Switzerland'
 'Portugal' 'Finland' 'Poland' 'Greece' 'Israel' 'Mexico' 'Slovenia'
 'China (People’s Republic of)' 'New Zealand' 'Japan' 'Chile' 'Germany'
 'Norway']
['Malignant neoplasms of female breast'
 'Malignant neoplasms of trachea, bronchus, lung' 'Malignant neoplasms'
 'Malignant neoplasms of colon, rectum and anus'
 'Malignant neoplasms of cervix uteri' 'Malignant neoplasms of prostate']


### Keep only total rows, sex specific cancer rows will have 'sex' field updated to say 'Total'

In [67]:


# Update fields in the "Sex" column to say "Total" for the sex specific cancers
OECD_cleaned.loc[OECD_cleaned['Cancer site'] == 'Malignant neoplasms of prostate', 'Sex'] = 'Total'
OECD_cleaned.loc[OECD_cleaned['Cancer site'] == 'Malignant neoplasms of female breast', 'Sex'] = 'Total'
OECD_cleaned.loc[OECD_cleaned['Cancer site'] == 'Malignant neoplasms of cervix uteri', 'Sex'] = 'Total'
# Print the updated DataFrame
print(OECD_cleaned)


              Country    Sex                                     Cancer site  \
0     Slovak Republic  Total            Malignant neoplasms of female breast   
1     Slovak Republic  Total            Malignant neoplasms of female breast   
2     Slovak Republic  Total            Malignant neoplasms of female breast   
3             Iceland   Male  Malignant neoplasms of trachea, bronchus, lung   
4             Iceland   Male  Malignant neoplasms of trachea, bronchus, lung   
...               ...    ...                                             ...   
1483      New Zealand  Total  Malignant neoplasms of trachea, bronchus, lung   
1484      Netherlands  Total   Malignant neoplasms of colon, rectum and anus   
1485      Netherlands  Total   Malignant neoplasms of colon, rectum and anus   
1486      Netherlands  Total   Malignant neoplasms of colon, rectum and anus   
1487      Netherlands  Total   Malignant neoplasms of colon, rectum and anus   

      Year  Cancer Rate  
0     2002   

### Keep only rows that are totals 

In [68]:
OECD_filtered = OECD_cleaned[OECD_cleaned['Sex'] == 'Total']

print(OECD_filtered)

              Country    Sex                                     Cancer site  \
0     Slovak Republic  Total            Malignant neoplasms of female breast   
1     Slovak Republic  Total            Malignant neoplasms of female breast   
2     Slovak Republic  Total            Malignant neoplasms of female breast   
6              Sweden  Total            Malignant neoplasms of female breast   
7              Sweden  Total            Malignant neoplasms of female breast   
...               ...    ...                                             ...   
1483      New Zealand  Total  Malignant neoplasms of trachea, bronchus, lung   
1484      Netherlands  Total   Malignant neoplasms of colon, rectum and anus   
1485      Netherlands  Total   Malignant neoplasms of colon, rectum and anus   
1486      Netherlands  Total   Malignant neoplasms of colon, rectum and anus   
1487      Netherlands  Total   Malignant neoplasms of colon, rectum and anus   

      Year  Cancer Rate  
0     2002   

In [69]:
# countries that that are missing too many years will have to be excluded for the model
# determine the number of entries per country 


# Group the DataFrame by 'Reference area' and count the number of entries for each group
entries_per_country = OECD_filtered.groupby(['Country']).size().reset_index(name='count')

# Print the result
print(entries_per_country)


                         Country  count
0                      Australia     18
1                        Austria     24
2                        Belgium     24
3                         Brazil      6
4                         Canada     18
5                          Chile     12
6   China (People’s Republic of)      6
7                       Colombia      6
8                     Costa Rica      6
9                        Czechia     18
10                       Denmark     24
11                       Estonia     12
12                       Finland     24
13                        France     24
14                       Germany     24
15                        Greece     24
16                       Hungary     18
17                       Iceland     18
18                         India      6
19                     Indonesia      6
20                       Ireland     24
21                        Israel     12
22                         Italy     24
23                         Japan     18


In [70]:
### Drop countries that have less than 18 entries

country_entry_counts = OECD_filtered['Country'].value_counts()

### Filter out countries with less than 18 entries
countries_to_keep = country_entry_counts[country_entry_counts >= 18].index.tolist()
OECD_final = OECD_filtered[OECD_filtered['Country'].isin(countries_to_keep)]

### Print the result
OECD_final.describe

<bound method NDFrame.describe of               Country    Sex                                     Cancer site  \
0     Slovak Republic  Total            Malignant neoplasms of female breast   
1     Slovak Republic  Total            Malignant neoplasms of female breast   
2     Slovak Republic  Total            Malignant neoplasms of female breast   
6              Sweden  Total            Malignant neoplasms of female breast   
7              Sweden  Total            Malignant neoplasms of female breast   
...               ...    ...                                             ...   
1483      New Zealand  Total  Malignant neoplasms of trachea, bronchus, lung   
1484      Netherlands  Total   Malignant neoplasms of colon, rectum and anus   
1485      Netherlands  Total   Malignant neoplasms of colon, rectum and anus   
1486      Netherlands  Total   Malignant neoplasms of colon, rectum and anus   
1487      Netherlands  Total   Malignant neoplasms of colon, rectum and anus   

     

In [71]:
#test that correct countries were dropped
# Group the DataFrame by 'Reference area' and count the number of entries for each group
entries_per_country2 = OECD_final.groupby(['Country']).size().reset_index(name='count')

# Print the result
print(entries_per_country2)


            Country  count
0         Australia     18
1           Austria     24
2           Belgium     24
3            Canada     18
4           Czechia     18
5           Denmark     24
6           Finland     24
7            France     24
8           Germany     24
9            Greece     24
10          Hungary     18
11          Iceland     18
12          Ireland     24
13            Italy     24
14            Japan     18
15            Korea     18
16       Luxembourg     24
17           Mexico     18
18      Netherlands     24
19      New Zealand     18
20           Norway     18
21           Poland     18
22         Portugal     24
23  Slovak Republic     18
24            Spain     24
25           Sweden     24
26      Switzerland     24
27          Türkiye     18
28   United Kingdom     24
29    United States     18


### Remove entires for the year 2000

In [72]:
# Remove all entries for the year 2000
OECD_final = OECD_final[OECD_final['Year'] != 2000]

# Print the result
OECD_final.describe


<bound method NDFrame.describe of               Country    Sex                                     Cancer site  \
0     Slovak Republic  Total            Malignant neoplasms of female breast   
1     Slovak Republic  Total            Malignant neoplasms of female breast   
2     Slovak Republic  Total            Malignant neoplasms of female breast   
7              Sweden  Total            Malignant neoplasms of female breast   
8              Sweden  Total            Malignant neoplasms of female breast   
...               ...    ...                                             ...   
1482      New Zealand  Total  Malignant neoplasms of trachea, bronchus, lung   
1483      New Zealand  Total  Malignant neoplasms of trachea, bronchus, lung   
1485      Netherlands  Total   Malignant neoplasms of colon, rectum and anus   
1486      Netherlands  Total   Malignant neoplasms of colon, rectum and anus   
1487      Netherlands  Total   Malignant neoplasms of colon, rectum and anus   

     

In [73]:
#drop sex feature as no longer necessary
OECD_final = OECD_final.drop(['Sex'], axis=1)

OECD_final.describe

<bound method NDFrame.describe of               Country                                     Cancer site  Year  \
0     Slovak Republic            Malignant neoplasms of female breast  2002   
1     Slovak Republic            Malignant neoplasms of female breast  2008   
2     Slovak Republic            Malignant neoplasms of female breast  2012   
7              Sweden            Malignant neoplasms of female breast  2002   
8              Sweden            Malignant neoplasms of female breast  2008   
...               ...                                             ...   ...   
1482      New Zealand  Malignant neoplasms of trachea, bronchus, lung  2008   
1483      New Zealand  Malignant neoplasms of trachea, bronchus, lung  2012   
1485      Netherlands   Malignant neoplasms of colon, rectum and anus  2002   
1486      Netherlands   Malignant neoplasms of colon, rectum and anus  2008   
1487      Netherlands   Malignant neoplasms of colon, rectum and anus  2012   

      Cancer Rate

### Export new dataframe to CSV

In [74]:
OECD_final.to_csv('OECD_Final.csv', index=False)